# Scraper para o site da Hemeroteca Digital de Lisboa

Esse notebook busca coletar os dados dos periódicos digitalizados e disponíveis no site da Hemeroteca Digital de Lisboa. Os dados serão organizados em um *dataframe* e exportados para um arquivo csv.

Os dados coletados são:

- **Título**: Título do periódico
- **Autoria**: Autoria do periódico
- **Período**: Período disponível do periódico
- **Link**: Link para o periódico
- **Fichas Históricas**: Link para as fichas históricas do periódico
- **Quantidade de PDF**: Quantidade de PDFs disponíveis do periódico
- **Links dos PDFs**: Lista com links para os PDFs disponíveis do periódico

In [1]:
# importar bibliotecas
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
# definir lista de letras que foram o índice
letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U','V', 'X', 'Z']

# definir url base
url_base = 'https://hemerotecadigital.cm-lisboa.pt/Indice/Indice'

# final da url
url_final = '.htm'

In [3]:
# definir lista de urls completa
urls = [url_base + letra + url_final for letra in letras]

In [4]:
# função para criar objeto soup
def criar_soup(url):
    html = requests.get(url)
    soup = bs(html.content, 'html.parser')
    return soup

In [5]:
# find all a href that ends with .htm and start with ../Periodicos or ../OBRAS
def encontrar_links(soup):
    links = soup.find_all('a', href=True)
    links = [link['href'] for link in links if link['href'].endswith('.htm') and (link['href'].startswith('../Periodicos') or link['href'].startswith('../OBRAS'))]
    # replace ../ with https://hemerotecadigital.cm-lisboa.pt/
    links = [link.replace('../', 'https://hemerotecadigital.cm-lisboa.pt/') for link in links]
    return links

In [6]:
# loop com as urls
links_geral = []
for url in urls:
    soup = criar_soup(url)
    links = encontrar_links(soup)
    links_geral.extend(links)

In [7]:
# função para pegar o título e a data de publicação
def pegar_titulo_data(soup):
    titulo_data = soup.find('title').text
    # encontrar data com regex: tudo entre [ e ]
    # se não encontrar [ no título, retorna None
    pattern = r'\['
    if re.search(pattern, titulo_data) == None:
        data = "Sem data"
        titulo = titulo_data
    else:
        data = re.findall(r'\[(.*?)\]', titulo_data)
        # encontrar título com regex: tudo antes de [
        titulo = re.findall(r'(.*?)\[', titulo_data)
    return titulo, data

In [8]:
# Encontrar todos os links que terminam em ".pdf"
def encontrar_pdf(soup, url):
    pdf_links = soup.find_all('a', href=True)
    pdf_links = [link['href'] for link in pdf_links if link['href'].endswith('.pdf') or link['href'].endswith('.PDF')]
    # excluir string após última "/" em url
    url = url.rsplit('/', 1)[0]
    url = url + '/'
    # add url to pdf_links
    pdf_links = [url + link for link in pdf_links]
    return pdf_links

In [9]:
# criar lista geral para os dados
dados_geral = []
# loop nos links
for link in links_geral:
    soup = criar_soup(link)
    titulo, data = pegar_titulo_data(soup)
    pdf_links = encontrar_pdf(soup, link)
    # se FichasHistoricas estiver no link, adicionar à lista de dados na variável fichas
    fichas = "Sem Ficha Histórica"
    if any("FichasHistoricas" in pdf for pdf in pdf_links):
        fichas = [pdf for pdf in pdf_links if "FichasHistoricas" in pdf]
        # correção do link para o pdf
        fichas = fichas[0].replace(link, 'https://hemerotecadigital.cm-lisboa.pt/')
    # contar o número de pdfs
    num_pdfs = len(pdf_links)
    # verificar se algum pdf possui a string"FichasHistoricas"
    # se sim, excluir da lista e subtrair 1 do número de pdfs
    if any("FichasHistoricas" in pdf for pdf in pdf_links):
        pdf_links = [pdf for pdf in pdf_links if "FichasHistoricas" not in pdf]
        num_pdfs = num_pdfs - 1
    # criar lista com os dados
    dados = [titulo, data, num_pdfs, pdf_links, fichas]
    # adicionar dados à lista de dados
    dados_geral.append(dados)


In [10]:
# criar dataframe com headers
df = pd.DataFrame(dados_geral, columns=['Título', 'Data', 'Quantidade de PDF', 'Links dos PDFs', 'Fichas Históricas'])

In [ ]:
# transformar itens da coluna 'Título' em string
df['Título'] = df['Título'].astype(str)
# remover colchetes e aspas da coluna 'Título'
df['Título'] = df['Título'].str.replace('[', '')
df['Título'] = df['Título'].str.replace(']', '')
df['Título'] = df['Título'].str.replace("'", '')
df['Título'] = df['Título'].str.replace('"', '')

In [ ]:
# transformar itens da coluna 'Data' em string
df['Data'] = df['Data'].astype(str)
# remover colchetes e aspas da coluna 'Data'
df['Data'] = df['Data'].str.replace('[', '')
df['Data'] = df['Data'].str.replace(']', '')
df['Data'] = df['Data'].str.replace("'", '')
df['Data'] = df['Data'].str.replace('"', '')


In [13]:
# se coluna data for vazia, preencher com "Sem data"
df['Data'] = df['Data'].replace('', 'Sem data')

Corrigindo títulos vazios

In [14]:
# se links_pdf não for vazio, pegar o primeiro link
if df['Links dos PDFs'].empty == False:
    # se coluna 'Título' for vazia, preencher com item da coluna 'Links dos PDFs'
    df.loc[df['Título'] == '', 'Título'] = df['Links dos PDFs'].str[0]

In [15]:
# transforma itens da coluna 'Título' em string
df['Título'] = df['Título'].astype(str)


In [ ]:
# excluir string até "Periodicos/" ou "OBRAS/" na coluna título usando regex
df['Título'] = df['Título'].str.replace(r'.*Periodicos/', '')
df['Título'] = df['Título'].str.replace(r'.*OBRAS/', '')
# substitui tudo após o primeiro "/" por vazio
df['Título'] = df['Título'].str.replace(r'/.*', '')

Excluir itens duplicados

In [17]:
# se valor na coluna 'Título' for repetido, exclui linha
df = df.drop_duplicates(subset=['Título'], keep='first')

Limpando as datas

In [18]:
# tente encontrar '\d{4}-\d{4}' na coluna 'Data' senão encontrar busca por '\d{4}'
df['Anos'] = df['Data'].str.findall(r'\d{4}-\d{4}|\d{4}')


In [ ]:
# deletar '\d{4}-\d{4}|\d{4}' da coluna 'Data'
df['Data'] = df['Data'].str.replace(r'\d{4}-\d{4}|\d{4}', '')
# deletar ';', '?', da coluna 'Data'
df['Data'] = df['Data'].str.replace(r';|\?', '')
# deletar 'Sem data' da coluna 'Data'
df['Data'] = df['Data'].str.replace(r'Sem data', '')
# deletar espaços em branco da coluna 'Data'
df['Data'] = df['Data'].str.strip()
# adicionar 'Sem local' à coluna 'Data' se estiver vazia
df['Data'] = df['Data'].replace('', 'Sem local')
# renomear coluna 'Data' para 'Local'
df = df.rename(columns={'Data': 'Local'})

In [20]:
# reorganizar colunas
df = df[['Título', 'Anos', 'Local', 'Fichas Históricas', 'Quantidade de PDF', 'Links dos PDFs']]

In [21]:
# exportar locais únicos para csv
df['Local'].value_counts().sort_values(ascending=False).to_csv('data/locais_hemeroteca_lisboa_raw.csv')

In [22]:
# manualmente corri os locais e criei o csv com os locais corretos
# leia o csv com os locais
locais = pd.read_csv('data/locais_hemeroteca_lisboa.csv', header=None)
# print o datafram locais
print(locais)

                                     0                        1
0                            Sem local                Sem local
1                              Lisboa,                   Lisboa
2                             Coimbra,                  Coimbra
3                      Rio de Janeiro,           Rio de Janeiro
4                               Porto,                    Porto
5                                    ,                Sem local
6                               Indice                Sem local
7                 Série 2 e 3, Lisboa,                   Lisboa
8               I e II Séries, Lisboa,                   Lisboa
9                             Novembro                Sem local
10            Lisboa e Rio de Janeiro,  Lisboa e Rio de Janeiro
11                            Porto, )                    Porto
12                              Praia,                    Praia
13          25 de Abril a 2 de Maio de                Sem local
14                  N.º 629 ao n.º 714  

In [23]:
# se o valor da coluna 0 no dataframe locais for igual ao valor da coluna 'Local' no dataframe df, substitua o valor da coluna 'Local' no dataframe df pelo valor da coluna 1 no dataframe locais
for i in range(len(locais)):
    df.loc[df['Local'] == locais[0][i], 'Local'] = locais[1][i]
    

In [24]:
df.head(20)

,Título,Anos,Local,Fichas Históricas,Quantidade de PDF,Links dos PDFs
0,Academia Portuguesa,[1933],Sem local,Sem Ficha Histórica,11,[https://hemerotecadigital.cm-lisboa.pt/Period...
1,O Académico: publicação bi-semanal redigida po...,[],Sem local,Sem Ficha Histórica,1,[https://hemerotecadigital.cm-lisboa.pt/Period...
2,O academico: revista quinzenal litteraria,[1878],Sem local,https://hemerotecadigital.cm-lisboa.pt/Periodi...,6,[https://hemerotecadigital.cm-lisboa.pt/Period...
4,O Académico: semanário ilustrado,[1902-1903],Sem local,https://hemerotecadigital.cm-lisboa.pt/Periodi...,8,[https://hemerotecadigital.cm-lisboa.pt/Period...
5,AccaoColonial_NComemorativo,[1934],Sem local,Sem Ficha Histórica,1,[https://hemerotecadigital.cm-lisboa.pt/Period...
6,Acção realista,[1924-1926],Sem local,Sem Ficha Histórica,29,[https://hemerotecadigital.cm-lisboa.pt/Period...
7,"La Acción, Ano 1, n.º 1 (1 de Febrero de 1908)",[],Sem local,Sem Ficha Histórica,1,[https://hemerotecadigital.cm-lisboa.pt/Period...
8,Actas da Câmara Municipal de Lisboa,[1974],Sem local,Sem Ficha Histórica,3,[https://hemerotecadigital.cm-lisboa.pt/OBRAS/...
9,Actas das sessões da Câmara Municipal de Lisboa,[],Sem local,Sem Ficha Histórica,8,[https://hemerotecadigital.cm-lisboa.pt/OBRAS/...
11,Água lustral : arte e crítica,[1913],Coimbra,Sem Ficha Histórica,1,[https://hemerotecadigital.cm-lisboa.pt/Period...


In [25]:
# salvar dataframe em csv
df.to_csv("../../../data/hml/hemeroteca_lisboa.csv", index=False)